In [ ]:
!pip install pyshp
!pip install geopandas
!pip install --upgrade nbformat
## Ensure running conda install --channel conda-forge cartopy before
!pip install geoplot


!pip install PyBNG


!pip install convertbng

In [ ]:
import geopandas as gpd
import glob
import pandas as pd
from convertbng.util import convert_bng, convert_lonlat



# Grid references for London: https://getoutside.ordnancesurvey.co.uk/guides/beginners-guide-to-grid-references/
# Read SHx files
files = [
#     "SP_RoadNode.shx",
#     "TL_RoadNode.shx",
#     "SU_RoadNode.shx",
#     "TQ_RoadNode.shx",
     "NC_RoadNode.shx"

]

path = "data_storage/data_storage/oproad_essh_gb/data/"

paths = [path+file for file in files]

geo_df = pd.concat(map(gpd.read_file, paths))



#lga_gdf = gpd.read_file('data_storage/data_storage/oproad_essh_gb/data/SP_RoadNode.shx')    # Load the data using Geopandas

In [ ]:
type(geo_df)

In [ ]:
geo_df.head()

In [ ]:
geo_df.dtypes

In [ ]:
geo_df["formOfNode"].unique()

In [ ]:
geo_df["formOfNode"].value_counts()

In [ ]:
geo_df.count()

### Conversion 

In [ ]:
import plotly.express as px
import geopandas as gpd
import plotly.io as pio
pio.renderers.default = 'browser'

sample = geo_df#.sample(n=1000)

sample["longitude"], sample["latitude"] = convert_lonlat(sample.geometry.x, sample.geometry.y)

In [ ]:
fig = px.scatter_mapbox(sample,
    lat=sample.latitude,
    lon=sample.longitude,
    hover_name="formOfNode",
    zoom=14
)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.write_html("output/road_features.html")
fig.show()

## Find nearest neighbour and distance from it

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree

## Create the two GeoDataFrame to replicate your dataset
appart = gpd.GeoDataFrame({
        'geometry': Point(a, b),
        'x': a,
        'y': b,
    } for a, b in zip(np.random.rand(100000), np.random.rand(100000))
)

pharma = gpd.GeoDataFrame([{
        'geometry': Point(a, b),
        'x': a,
        'y': b,
    } for a, b in zip(np.random.rand(3000), np.random.rand(3000))
])

# Create a BallTree 
tree = BallTree(pharma[['x', 'y']].values, leaf_size=2)

# Query the BallTree on each feature from 'appart' to find the distance
# to the nearest 'pharma' and its id
appart['distance_nearest'], appart['id_nearest'] = tree.query(
    appart[['x', 'y']].values, # The input array for the query
    k=1, # The number of nearest neighbors
)


In [ ]:
# Python3 implementation of the approach
from math import sqrt
 
# Function to find the circle on
# which the given three points lie
def findCircle(x1, y1, x2, y2, x3, y3) :
    x12 = x1 - x2;
    x13 = x1 - x3;
 
    y12 = y1 - y2;
    y13 = y1 - y3;
 
    y31 = y3 - y1;
    y21 = y2 - y1;
 
    x31 = x3 - x1;
    x21 = x2 - x1;
 
    # x1^2 - x3^2
    sx13 = pow(x1, 2) - pow(x3, 2);
 
    # y1^2 - y3^2
    sy13 = pow(y1, 2) - pow(y3, 2);
 
    sx21 = pow(x2, 2) - pow(x1, 2);
    sy21 = pow(y2, 2) - pow(y1, 2);
 
    f = (((sx13) * (x12) + (sy13) *
          (x12) + (sx21) * (x13) +
          (sy21) * (x13)) / (2 *
          ((y31) * (x12) - (y21) * (x13))));
             
    g = (((sx13) * (y12) + (sy13) * (y12) +
          (sx21) * (y13) + (sy21) * (y13)) /
          (2 * ((x31) * (y12) - (x21) * (y13))));
 
    c = (-pow(x1, 2) - pow(y1, 2) -
         2 * g * x1 - 2 * f * y1);
 
    # eqn of circle be x^2 + y^2 + 2*g*x + 2*f*y + c = 0
    # where centre is (h = -g, k = -f) and
    # radius r as r^2 = h^2 + k^2 - c
    h = -g;
    k = -f;
    sqr_of_r = h * h + k * k - c;
 
    # r is the radius
    r = round(sqrt(sqr_of_r), 5);
 
    print("Centre = (", h, ", ", k, ")");
    print("Radius = ", r);
 
# Driver code
if __name__ == "__main__" :
     
    x1 = 1 ; y1 = 1;
    x2 = 2 ; y2 = 4;
    x3 = 5 ; y3 = 3;
    findCircle(x1, y1, x2, y2, x3, y3);
 
# This code is contributed by Ryuga